In [8]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-131781")
exp = Experiment(workspace=ws, name="hyperDrive-pipeline-project1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-131781
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-131781


In [51]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

compute_cluster = "hyper-p1"

try:
    compute_target = ComputeTarget(ws, compute_cluster)
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_cluster, config)
  

In [47]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

compute-instance-p1 ComputeInstance Succeeded
hyper-p1 AmlCompute Succeeded


In [70]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.core import ScriptRunConfig
import os

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling({
        '--C': choice(0.01, 0.1, 1, 10, 100, 1000), 
        '--max_iter': choice(25, 50, 100, 200)
    })

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=3)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
# est = ScriptRunConfig(source_directory="./", compute_target=compute_target, script='train.py')
est =SKLearn(source_directory=".",compute_target=compute_target,entry_script='train.py')
# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig( hyperparameter_sampling = ps, primary_metric_name='Accuracy', primary_metric_goal= PrimaryMetricGoal.MAXIMIZE, policy = policy, max_total_runs = 10, max_concurrent_runs = 4, estimator=est)

In [74]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails
hyperdrive_run = exp.submit(hyperdrive_config,show_output=True)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [75]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_ecd15d1e-0e75-448c-a023-2ba7dee7125a
Web View: https://ml.azure.com/experiments/hyperDrive-pipeline-project1/runs/HD_ecd15d1e-0e75-448c-a023-2ba7dee7125a?wsid=/subscriptions/a24a24d5-8d87-4c8a-99b6-91ed2d2df51f/resourcegroups/aml-quickstarts-131781/workspaces/quick-starts-ws-131781

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-12-26T00:41:22.759623][API][INFO]Experiment created<END>\n""<START>[2020-12-26T00:41:23.319931][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2020-12-26T00:41:23.695373][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2020-12-26T00:41:24.1061345Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END><START>[2020-12-26T00:41:54.7941593Z][SCHEDULER][INFO]Scheduling job, id='HD_ecd15d1e-0e75-448c-a023-2ba7dee7125a_0'<END><START>[2020-12-26T00:41:54.7954704Z][SCHEDULER][INFO]Scheduli

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
# best_run_metrics = best_run.get_metrics()
# os.makedirs('outputs', exist_ok=True)
# joblib.dump(value= best_run.id, filename = 'outputs/model.joblib')
# best_run.get_file_names()
model = best_run.register_model(model_name='best_model_p1',model_path='outputs/model.joblib')

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_p1 = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [6]:
import sys
sys.path.insert(0, './training')
from train import clean_data

# Use the clean_data function to clean your data.
### YOUR DATA OBJECT HERE ###
x, y = clean_data(data_p1)

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state = 42, test_size = 0.2)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

NameError: name 'x' is not defined

In [2]:
import pandas as pd

data_train= pd.concat([x_train, y_train], axis=1)

NameError: name 'x_train' is not defined

In [1]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=data_train,
    label_column_name='y',
    n_cross_validations=4)

NameError: name 'data_train' is not defined

In [3]:
# Submit your automl run
from azureml.core import Workspace, Experiment
from azureml.pipeline.steps import AutoMLStep
### YOUR CODE HERE ###
ws = Workspace.get(name="quick-starts-ws-131781")
exp = Experiment(ws, "automl-exp")
automl_run = exp.submit(automl_config, show_output=True)
# automl_run.wait_for_completion()

NameError: name 'automl_config' is not defined

In [105]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_automl_run, best_model = automl_run.get_output()
best_automl_run.register_model(model_name = "automl_model_p1.pkl", model_path = './outputs/')

NameError: name 'automl_run' is not defined

In [ ]:
compute_target.delete()